# Data Science v cloudu: Cesta s "Azure ML SDK"

## Úvod

V tomto notebooku se naučíme, jak používat Azure ML SDK k trénování, nasazení a využití modelu prostřednictvím Azure ML.

Předpoklady:
1. Vytvořili jste pracovní prostor Azure ML.
2. Načetli jste [dataset Heart Failure](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) do Azure ML.
3. Nahráli jste tento notebook do Azure ML Studio.

Další kroky jsou:

1. Vytvořit experiment v existujícím pracovním prostoru.
2. Vytvořit výpočetní cluster.
3. Načíst dataset.
4. Nakonfigurovat AutoML pomocí AutoMLConfig.
5. Spustit experiment AutoML.
6. Prozkoumat výsledky a získat nejlepší model.
7. Zaregistrovat nejlepší model.
8. Nasadit nejlepší model.
9. Využít endpoint.

## Specifické importy pro Azure Machine Learning SDK


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## Inicializace pracovního prostoru
Inicializujte objekt pracovního prostoru z uložené konfigurace. Ujistěte se, že konfigurační soubor je přítomen v .\config.json


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Vytvoření experimentu Azure ML

Vytvořme experiment s názvem 'aml-experiment' v pracovním prostoru, který jsme právě inicializovali.


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## Vytvoření výpočetního clusteru
Budete muset vytvořit [výpočetní cíl](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) pro váš AutoML běh.


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## Data  
Ujistěte se, že jste nahráli datovou sadu do Azure ML a že klíč má stejný název jako datová sada.


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

## Konfigurace AutoML


In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## Spuštění AutoML


In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## Nasazení nejlepšího modelu

Spusťte následující kód pro nasazení nejlepšího modelu. Stav nasazení můžete sledovat v portálu Azure ML. Tento krok může trvat několik minut.


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## Použití koncového bodu
Můžete přidat vstupy do následujícího vzorového vstupu.


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**Prohlášení**:  
Tento dokument byl přeložen pomocí služby pro automatický překlad [Co-op Translator](https://github.com/Azure/co-op-translator). Ačkoli se snažíme o přesnost, mějte prosím na paměti, že automatické překlady mohou obsahovat chyby nebo nepřesnosti. Původní dokument v jeho původním jazyce by měl být považován za autoritativní zdroj. Pro důležité informace se doporučuje profesionální lidský překlad. Neodpovídáme za žádné nedorozumění nebo nesprávné interpretace vyplývající z použití tohoto překladu.
